In [9]:
!pip install beautifulsoup4

In [8]:
!pip install requests

In [4]:
!/Users/ali/Documents/IMDB/myenv/bin/python3 -m pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 322 kB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


In [87]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 1.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 1.5 MB/s eta 0:00:0000:0100:01


In [6]:
!pip install --upgrade pip

In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
link='https://www.imdb.com/chart/top/'
x = requests.get(link)
soup = BeautifulSoup(x.content, 'html.parser')
list_links=[i.find('a') for i in soup.find_all('tr')[1:]]
main_link='https://www.imdb.com'
all_links=[[main_link+i.find('a')['href'],i.find('a').img['alt']] for i in soup.find_all('tr')[1:]]
all_links[0]

['https://www.imdb.com/title/tt0111161/', 'The Shawshank Redemption']

In [4]:
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)


In [7]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

In [94]:
from tqdm import tqdm
import pandas as pd


movie_df = pd.DataFrame(columns=['movie_id', 'title', 'year', 'parental_guide','runtime','storyline','gross_us_canada'])
genres_df = pd.DataFrame(columns=['genre_name'])
movie_genres_df = pd.DataFrame(columns=['movie_id', 'genre_name'])
movie_directors_df = pd.DataFrame(columns=['movie_id', 'director_id'])
movie_writer_df = pd.DataFrame(columns=['movie_id', 'writer_name'])
movie_stars_df = pd.DataFrame(columns=['movie_id', 'stars_name'])



for i in tqdm(range(len(all_links))):
    print(all_links[i])
    x=requests.get(all_links[i][0], headers=HEADERS)
    movie_id=all_links[i][0].split('/')[-2]
    soup = BeautifulSoup(x.content, 'html.parser')
    title = soup.find('span',{'class':'sc-afe43def-1 fDTGTb'}).text
    year=soup.find('div',{'class':'sc-52d569c6-0'}).find_all('a',{'class','ipc-link ipc-link--baseAlt ipc-link--inherit-color'})[0].text
    try:
        parental_guide=soup.find('div',{'class':'sc-52d569c6-0'}).find_all('a',{'class','ipc-link ipc-link--baseAlt ipc-link--inherit-color'})[1].text
    except :
        parental_guide='Nan'
    runtime=0
    try:
        runtime=runtime+int(soup.find('div',{'class':'sc-52d569c6-0'}).find_all('li',{'class','ipc-inline-list__item'})[-1].text.split('h')[0])*60
        runtime=runtime+int(soup.find('div',{'class':'sc-52d569c6-0'}).find_all('li',{'class','ipc-inline-list__item'})[-1].text.split('h')[1].split('m')[0])
    except :
        pass
    genres=[ganre.text for ganre in soup.find_all('a',{'class':'ipc-chip ipc-chip--on-baseAlt'})]
    
    try:
        listo=[row.find('div',{'class':'ipc-metadata-list-item__content-container'}).find_all('li',{'class':'ipc-inline-list__item'}) for row in  soup.find('section',{'class','sc-52d569c6-4 eAiKYC'}).find_all('li',{'class':'ipc-metadata-list__item'})[:-1]]
    except :
        listo=[row.find('div',{'class':'ipc-metadata-list-item__content-container'}).find_all('li',{'class':'ipc-inline-list__item'}) for row in  soup.find('section',{'class','sc-52d569c6-4 bTarzB'}).find_all('li',{'class':'ipc-metadata-list__item'})[:-1]]
    
    list_all=[]
    for et in listo:
        arr=[]
        for at in et:
            arr.append(at.text)
        list_all.append(arr)
    directors=list_all[0]
    writers=list_all[1]
    stars=list_all[2]
    box_ofices=soup.find_all('li',{'class':'ipc-metadata-list__item sc-6d4f3f8c-2 byhjlB'})
    gross_us_canada='Nan'
    for box in box_ofices:
        if box.find('span',{'class':'ipc-metadata-list-item__label'}).text=='Gross US & Canada':
            gross_us_canada=box.find('div',{'class':'ipc-metadata-list-item__content-container'}).text
            break 
    params = {
    'operationName': 'TMD_Storyline',
    'variables':'{"locale":"en-US","titleId":"'+movie_id+'"}',
    'extensions':'{"persistedQuery":{"sha256Hash":"87f41463a48af95ebba3129889d17181402622bfd30c8dc9216d99ac984f0091","version":1}}'

     }
    headers = {'Content-type': 'application/json'}
    resp = requests.get('https://caching.graphql.imdb.com', params=params,headers=headers)
    storyline=resp.json()['data']['title']['summaries']['edges'][0]['node']['plotText']['plaidHtml']
    new_row = pd.DataFrame({'movie_id': [movie_id], 'title': [title], 'year': [year], 'parental_guide': [parental_guide],
                        'runtime': [runtime], 'storyline': [storyline], 'gross_us_canada': [gross_us_canada]})
    movie_df = pd.concat([movie_df, new_row], ignore_index=True)
    

  0%|                                                   | 0/250 [00:00<?, ?it/s]

['https://www.imdb.com/title/tt0111161/', 'The Shawshank Redemption']


  0%|▏                                          | 1/250 [00:02<10:45,  2.59s/it]

['https://www.imdb.com/title/tt0068646/', 'The Godfather']


  1%|▎                                          | 2/250 [00:05<12:14,  2.96s/it]

...

 99%|████████████████████████████████████████▋| 248/250 [15:01<00:09,  4.80s/it]

['https://www.imdb.com/title/tt0099348/', 'Dances with Wolves']


100%|████████████████████████████████████████▊| 249/250 [15:05<00:04,  4.58s/it]

['https://www.imdb.com/title/tt0071411/', 'Dersu Uzala']


100%|█████████████████████████████████████████| 250/250 [15:08<00:00,  3.63s/it]


In [95]:
movie_df

,id,title,year,parental_guid,runtime,storyline,gross_us_canada,movie_id,parental_guide
0,NaN,The Shawshank Redemption,1994,NaN,142,Chronicles the experiences of a formerly succe...,"$28,767,189",tt0111161,R
1,NaN,The Godfather,1972,NaN,175,The Godfather &quot;Don&quot; Vito Corleone is...,"$136,381,073",tt0068646,R
2,NaN,The Dark Knight,2008,NaN,152,Set within a year after the events of Batman B...,"$534,987,076",tt0468569,PG-13
3,NaN,The Godfather Part II,1974,NaN,202,The continuing saga of the Corleone crime fami...,"$47,834,595",tt0071562,R
4,NaN,12 Angry Men,1957,NaN,96,"The defense and the prosecution have rested, a...",Nan,tt0050083,Approved
...,...,...,...,...,...,...,...,...,...
245,NaN,The Iron Giant,1999,NaN,86,This is the story of a nine-year-old boy named...,"$23,315,035",tt0129167,PG
246,NaN,The Help,2011,NaN,146,"Set in Mississippi during the 1960s, Skeeter (...","$169,708,112",tt1454029,PG-13
247,NaN,Aladdin,1992,NaN,90,Aladdin is a poor street urchin who spends his...,"$217,350,219",tt0103639,G
248,NaN,Dances with Wolves,1990,NaN,181,Lt. John Dunbar is dubbed a hero after he acci...,"$184,208,848",tt0099348,PG-13


In [96]:
movie_df.to_csv('movie_df.csv', index=False)

In [62]:
params = {
    'operationName': 'TMD_Storyline',
    'variables':'{"locale":"en-US","titleId":"tt0068646"}',
    'extensions':'{"persistedQuery":{"sha256Hash":"87f41463a48af95ebba3129889d17181402622bfd30c8dc9216d99ac984f0091","version":1}}'

}
headers = {'Content-type': 'application/json'}
resp = requests.get('https://caching.graphql.imdb.com', params=params,headers=headers)

In [72]:
resp.json()['data']['title']['summaries']['edges'][0]['node']['plotText']['plaidHtml']

'The Godfather &quot;Don&quot; Vito Corleone is the head of the Corleone mafia family in New York. He is at the event of his daughter&#39;s wedding. Michael, Vito&#39;s youngest son and a decorated WW II Marine is also present at the wedding. Michael seems to be uninterested in being a part of the family business. Vito is a powerful man, and is kind to all those who give him respect but is ruthless against those who do not. But when a powerful and treacherous rival wants to sell drugs and needs the Don&#39;s influence for the same, Vito refuses to do it. What follows is a clash between Vito&#39;s fading old values and the new ways which may cause Michael to do the thing he was most reluctant in doing and wage a mob war against all the other mafia families which could tear the Corleone family apart.'